In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from corr_parc import partial_correlation
from merge import generate_merged_file

In [2]:
df_merged = generate_merged_file()

In [3]:
df_merged.columns

Index(['Date', 'energy_price', 'brent_value', 'precipitacion_unidadmedida',
       'precipitacion_amazonas', 'precipitacion_arauca',
       'precipitacion_atlantico', 'precipitacion_bogota',
       'precipitacion_bolivar', 'precipitacion_caqueta',
       'precipitacion_casanare', 'precipitacion_cesar', 'precipitacion_choco',
       'precipitacion_guainia', 'precipitacion_guaviare',
       'precipitacion_la guajira', 'precipitacion_magdalena',
       'precipitacion_meta', 'precipitacion_narino',
       'precipitacion_norte de santander', 'precipitacion_putumayo',
       'precipitacion_quindio', 'precipitacion_risaralda',
       'precipitacion_san andres providencia', 'precipitacion_sucre',
       'precipitacion_vaupes', 'precipitacion_vichada',
       'precipitacion_departamentos_represa', 'temp_AMAZONAS', 'temp_ARAUCA',
       'temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
       'temp_ATLANTICO', 'temp_BOGOTA', 'temp_BOLIVAR', 'temp_CAQUETA',
       'temp_CASANARE', '

In [4]:
df_merged.shape

(644, 52)

In [5]:
print(f"Fecha mínima: {df_merged.loc[0]['Date']}")

Fecha mínima: 2021-10-01


In [6]:
print(f"Fecha máxima: {df_merged.loc[df_merged.shape[0] - 1]['Date']}")

Fecha máxima: 2023-09-30


In [7]:
df_merged.describe()

,energy_price,brent_value,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_CHOCO,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas,TRM
count,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000,644.000000
mean,322.121164,89.611661,6.400516,10.241483,40.353079,10.732652,47.941181,17.652089,63.382711,23.383042,879.326638,7.906128,5.772768,12.481480,24.702544,38.097478,46.312276,62.416452,12.916026,26.480121,36.306591,8.282188,22.101696,5.926174,7.853591,670.819349,29.808241,6.905055,30.566177,32.733805,18.975474,32.087316,28.876018,30.101378,32.095867,27.480361,32.253339,31.809686,32.101577,30.685036,27.907349,20.107569,25.823510,27.823546,26.534008,26.457596,32.580889,30.547821,24.899482,4313.503168
std,241.618420,12.144518,10.791224,15.760833,92.093514,18.775336,75.967073,33.094526,82.616821,58.539379,328.149337,12.940953,8.245064,31.525180,37.093867,65.005119,59.017196,90.121622,19.851035,41.786594,59.749933,20.246645,24.252710,4.760183,18.704236,525.399107,2.256749,4.051862,1.312566,1.863389,2.125882,1.863505,3.035518,2.246416,2.438218,2.722574,2.363935,1.340409,1.787802,1.590859,2.306036,2.133082,3.644461,2.869805,2.950855,2.582673,2.228982,2.634650,1.997670,374.097100
min,87.255265,70.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,9.060000,18.500000,0.900000,26.400000,24.400000,8.800000,23.975000,17.255950,20.874515,18.779405,16.812520,23.200000,26.100000,22.342857,22.000000,21.044440,7.600000,15.055556,11.953600,15.400000,13.733333,24.333333,22.800000,15.752039,3706.950000
25%,133.920626,80.808750,0.119000,0.000000,5.000000,0.435000,5.080000,0.200000,9.565000,0.200000,873.612678,0.100000,0.515000,0.400000,1.100000,2.242000,4.778250,2.115000,0.600000,0.447750,2.288500,0.000000,5.100000,1.780000,0.000000,258.800500,28.795000,4.440000,29.760000,31.972917,17.942312,31.120000,26.985175,29.140610,30.882789,25.913975,31.250000,31.315000,31.150000,29.839286,26.585867,19.082206,23.635557,26.500000,24.820119,25.166860,31.490000,28.285000,23.990580,3972.902500
50%,239.984244,85.955000,2.367000,3.050000,16.125000,3.925000,19.515000,5.250000,38.023000,3.700000,1014.850000,3.190000,4.000000,2.125000,10.900000,13.250000,26.535000,21.268000,6.970000,8.739500,16.728000,0.800000,15.400000,4.720000,0.400000,548.155000,30.100000,5.800000,30.497094,32.900000,19.019167,32.245000,28.907528,30.293747,32.345813,27.599233,32.500000,32.120000,32.182141,30.776465,28.069519,20.307821,25.107536,28.117632,26.294401,26.476529,33.000000,30.900000,24.913238,4303.340000
75%,435.658130,95.925000,9.266018,1

### Corrleción variables explicativas con variable dependiente con data estándarizada


In [8]:
scaler = StandardScaler()
df_to_corr = df_merged.drop(
    columns={"Date", "precipitacion_unidadmedida", "energy_price"}
)
scaled_df_complete = scaler.fit_transform(df_to_corr)
scaled_df_complete = pd.DataFrame(scaled_df_complete, columns=df_to_corr.columns)
scaled_df_complete["energy_price"] = df_merged["energy_price"]
# scaled_df_complete.corr().style.background_gradient(cmap="coolwarm",vmin=-1,vmax=1)
scaled_df_complete.corr()[scaled_df_complete.corr().shape[0] - 1 :].T.sort_values(
    by="energy_price", ascending=False
).style.background_gradient(cmap="coolwarm", vmin=-1, vmax=1)

,energy_price
energy_price,1.000000
temp_NORTE DE SANTANDER,0.576889
temp_QUINDIO,0.461427
temp_avg_represas,0.448470
temp_CESAR,0.402348
temp_BOGOTA,0.384643
temp_BOLIVAR,0.324716
temp_GUAVIARE,0.321827
temp_ATLANTICO,0.318033
temp_MAGDALENA,0.309976


### Correlación Parcial de X con y


In [9]:
partial_correlations = []
variables = []

for x in list(scaled_df_complete.drop(columns=["energy_price"]).columns):
    temp_r = partial_correlation(
        scaled_df_complete,
        x,
        "energy_price",
        list(scaled_df_complete.drop(columns=["energy_price", x])),
    )["r"].iloc[0]
    partial_correlations.append(temp_r)
    variables.append(x)

df_corr_parc = pd.DataFrame(
    {"Variable": variables, "partial_correlation": partial_correlations}
)
df_corr_parc["abs_partial_correlation"] = df_corr_parc["partial_correlation"].abs()
df_corr_parc.sort_values(by="abs_partial_correlation", ascending=False)[
    ["Variable", "abs_partial_correlation"]
].style.background_gradient(cmap="coolwarm", vmin=-1, vmax=1)

,Variable,abs_partial_correlation
48,TRM,0.205374
0,brent_value,0.198904
41,temp_NORTE DE SANTANDER,0.195440
31,temp_CAQUETA,0.168733
33,temp_CESAR,0.158519
40,temp_NARINO,0.133458
44,temp_RISARALDA,0.128130
35,temp_GUAINIA,0.118563
47,temp_avg_represas,0.115584
37,temp_LA GUAJIRA,0.114014


### Correlación entre variables explicativas.

Se analiza con el objetivo de quitar variables altamente correlacionadas para evitar multicolinealidad.


In [10]:
precipitacion_columns = df_merged.filter(like="precipitacion_")
temp_columns = df_merged.filter(like="temp_")
specific_columns_df = df_merged[["brent_value", "TRM"]]
exogen_data = pd.concat(
    [precipitacion_columns, temp_columns, specific_columns_df], axis=1
)
exogen_data.drop("precipitacion_unidadmedida", axis=1, inplace=True)

## Estandarización de la data

Para empezar a modelar la data, se debe estandarizar los datos, para ello se utilizará la librería `sklearn.preprocessing`, la cual nos permite estandarizar los datos de una manera sencilla.


In [11]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(exogen_data)
scaled_df = pd.DataFrame(scaled_data, columns=exogen_data.columns)
scaled_df

,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_CHOCO,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas,brent_value,TRM
0,-0.509005,0.759339,0.139390,-0.557688,4.888002,-0.530775,-0.284457,-0.030483,0.599807,-0.518613,-0.661851,0.194869,1.081003,-0.330961,0.900229,-0.425494,-0.641070,-0.514443,-0.197754,-0.290752,0.367185,1.680616,-0.382756,-0.103180,0.550670,3.036752,0.940739,-0.367254,-0.129682,-0.395968,0.503965,0.536433,-0.100627,0.994447,0.189095,-0.081894,0.502920,-0.340075,0.777978,1.267949,0.196452,0.584624,-0.132406,0.000597,1.422880,0.323702,-0.114337,-0.917312,-1.339552
1,-0.593583,1.622908,-0.161734,-0.569948,5.703726,-0.437031,-0.355927,0.273823,1.312841,-0.217009,0.073705,0.296772,0.108929,-0.280157,-0.061255,-0.533210,-0.651153,0.563295,-0.023560,-0.206723,0.358932,0.541119,-0.356003,0.217245,0.750227,3.234346,0.864493,0.706894,0.858911,0.926494,0.594068,0.957301,0.486227,0.773896,0.908794,0.590065,0.558898,0.233086,1.342154,1.044002,0.036264,0.758987,-0.077677,0.701471,0.203140,1.159376,0.619803,-0.917312,-1.423606
2,-0.538867,1.559410,-0.248126,-0.567816,3.445073,-0.527751,-0.198452,-0.324530,0.818475,-0.402611,-0.450652,-0.226074,0.036084,-0.400241,-0.445679,-0.258923,-0.615863,-0.138432,-0.569591,0.248027,0.433208,1.436738,-0.420209,-0.115418,0.359982,3.357842,0.902616,0.371223,1.235518,0.704964,0.670510,0.919653,0.103689,1.083718,1.162805,1.247091,1.006723,0.079311,1.304955,1.926640,0.207090,1.561059,-0.136915,0.869386,1.939212,1.159376,0.950203,-0.917312,-1.423606
3,-0.580600,1.718155,-0.183251,-0.287443,3.802742,-0.300951,0.339387,-0.336497,0.724237,-0.472212,-0.159342,-0.231788,0.155604,0.027750,0.233125,-0.288906,-0.504952,-0.294105,-0.455695,0.416086,0.016436,0.873297,0.532186,0.289020,-0.606760,3.370191,0.063912,0.438357,1.000139,0.711677,0.470931,0.532741,0.215533,1.293766,0.316100,0.530335,0.969404,0.149209,1.168561,1.612333,0.213532,0.863605,0.110949,0.588202,2.107581,0.057805,0.723472,-0.761153,-1.423606
4,0.016831,2.981760,0.521690,-0.560886,0.362265,0.158697,-0.180160,0.179796,0.586998,-0.565013,0.741289,-0.272739,2.881082,2.346292,0.199601,0.248564,1.188971,-0.444989,-0.325049,-0.266038,0.338299,2.731812,-0.281095,1.137304,-0.424941,3.036752,0.407018,0.424930,-0.600440,0.939920,-0.016688,-0.130093,-0.205928,-0.418129,0.612447,-0.380542,0.876107,0.198137,0.430792,0.717358,-0.146140,-0.008211,-1.183151,-0.106008,-0.365573,-0.701899,-0.197147,-0.614059,-1.411032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,0.847595,-0.631262,-0.323217,-0.572080,0.542219,-0.003994,-0.336545,-0.374108,0.015473,-0.611413,-0.688555,-0.360674,-0.178134,-0.548344,-0.686151,-0.692008,-0.651153,-0.634192,-0.608115,-0.053492,2.665620,0.141665,-0.420209,-0.607209,-0.336249,-0.050647,2.541903,0.827735,2.177035,0.078417,1.821197,2.471909,0.466878,-0.642181,1.713163,1.097767,1.146668,1.645021,0.168151,-0.034829,2.918058,1.665677,1.158528,1.062673,-0.530200,1.767140,1.666261,0.279630,-0.698124
640,-0.593583,-0.555064,-0.213895,0.936388,-0.232402,-0.066289,-0.

In [12]:
scaled_df.cov()

,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_CHOCO,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas,brent_value,TRM
precipitacion_amazonas,1.001555,-0.066191,-0.075131,0.106628,-0.033421,-0.022749,-0.013532,-0.015310,0.083181,-0.059639,0.043743,-0.105627,-0.001359,-0.022341,0.036714,0.046925,-0.000846,0.066826,0.028295,-0.062579,0.008749,0.003205,0.008486,0.063866,-0.232876,0.073633,-0.060187,0.110502,0.052563,0.082170,0.055560,0.060612,0.086185,0.127657,0.101707,0.026660,0.017859,-0.012005,0.122950,0.102052,-0.008484,0.041793,-0.011271,0.026760,0.104096,-0.016484,0.036455,0.015203,-0.011634
precipitacion_arauca,-0.066191,1.001555,0.017738,0.045786,0.247773,0.209618,0.208907,0.061259,-0.017647,0.108003,0.117867,0.039150,0.109301,0.248046,-0.007042,-0.047489,0.056551,0.031084,0.097037,0.142430,0.190364,0.131184,0.287807,0.244763,0.038574,0.380642,0.185850,-0.041552,-0.123576,-0.047016,-0.208424,-0.149702,-0.063187,-0.109282,-0.071320,-0.242912,0.161232,0.006812,-0.131230,0.084998,0.017754,-0.094839,-0.038259,0.000819,0.021908,0.022720,-0.065313,0.034978,-0.261270
precipitacion_atlantico,-0.075131,0.017738,1.001555,0.013373,0.119354,0.019560,0.052925,0.036554,0.085810,0.116756,0.119390,0.230659,0.204397,0.060767,0.057966,0.053579,0.012269,0.022570,0.051110,0.138086,0.114391,-0.050901,-0.019457,0.163645,0.133744,-0.045925,0.118015,-0.168461,0.006943,-0.165249,0.070278,0.007887,-0.124928,-0.181289,0.068925,-0.023144,-0.049412,0.028467,0.089315,0.041169,-0.052099,0.087134,-0.092169,-0.067516,-0.184989,0.095596,-0.082364,0.115646,-0.007311
precipitacion_bogota,0.106628,0.045786,0.013373,1.001555,0.074311,0.104891,0.118155,0.109288,-0.002046,0.059356,0.216080,0.002579,0.242559,0.286958,0.286281,0.110295,0.033573,0.332831,0.236010,-0.014214,0.075406,0.073002,0.037746,0.435088,-0.056745,-0.061703,-0.010437,-0.036821,-0.116358,-0.068289,-0.121273,-0.107366,-0.086043,-0.045275,-0.034680,-0.205622,0.001196,-0.045062,-0.076284,0.033023,-0.155419,0.010909,-0.152240,-0.101085,-0.036298,-0.082724,-0.158058,0.252581,-0.042178
precipitacion_bolivar,-0.033421,0.247773,0.119354,0.074311,1.001555,0.063905,0.092061,0.142962,-0.004285,0.073235,0.168201,0.138465,0.236089,0.089552,0.090854,0.041116,-0.079415,0.048753,0.130668,0.120675,0.341056,0.145689,0.047147,0.252758,0.007806,0.159235,0.003813,-0.218968,-0.121543,-0.244000,-0.077200,-0.091902,-0.218700,-0.152654,-0.025572,-0.209844,-0.010259,-0.033634,-0.007331,0.043443,-0.110220,-0.008467,-0.202183,-0.137568,-0.192047,0.036665,-0.199382,0.191432,-0.167001
precipitacion_caqueta,-0.022749,0.209618,0.019560,0.104891,0.063905,1.001555,0.229496,0.085323,-0.036160,0.146705,0.223487,-0.008890,0.110940,0.417747,0.149589,-0.000407,0.299479,0.024059,0.092549,0.127396,0.066655,0.079373,0.118988,0.214015,-0.023799,0.012242,0.018268,-0.053754,-0.118165,-0.081272,-0.299286,-0.139331,-0.066828,-0.104124,-0.060445,-0.249987,0.033005,0.093049,-0.182871,-0.056930,0.004889,-0.195668,-0.056701,-0.046964,-0.062599,-0.034148,-0.112516,0.051162,-0.007921
precipitacion_casanare,-0.013532,0.208907,0.052925,0.118155,0.092061,0.229496,1.001555,0.018476,0.199215,0.142158,0.223643,0.023070,0.122934,0.234588,-

In [13]:
scaled_df_corr = scaled_df.corr()
print("Correlation Matrix Original: ", scaled_df_corr.shape)
scaled_df_corr.style.background_gradient(cmap="coolwarm")

Correlation Matrix Original:  (49, 49)


,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_CHOCO,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas,brent_value,TRM
precipitacion_amazonas,1.000000,-0.066088,-0.075014,0.106463,-0.033369,-0.022714,-0.013511,-0.015286,0.083052,-0.059546,0.043675,-0.105463,-0.001357,-0.022306,0.036657,0.046852,-0.000844,0.066723,0.028251,-0.062482,0.008736,0.003200,0.008473,0.063766,-0.232514,0.073518,-0.060093,0.110331,0.052482,0.082042,0.055474,0.060518,0.086051,0.127459,0.101549,0.026618,0.017831,-0.011986,0.122759,0.101894,-0.008471,0.041728,-0.011253,0.026718,0.103934,-0.016458,0.036399,0.015180,-0.011616
precipitacion_arauca,-0.066088,1.000000,0.017710,0.045715,0.247388,0.209292,0.208583,0.061164,-0.017620,0.107836,0.117684,0.039089,0.109131,0.247661,-0.007031,-0.047415,0.056463,0.031036,0.096886,0.142209,0.190068,0.130980,0.287361,0.244383,0.038514,0.380051,0.185561,-0.041487,-0.123384,-0.046943,-0.208100,-0.149469,-0.063089,-0.109112,-0.071209,-0.242535,0.160982,0.006801,-0.131026,0.084866,0.017726,-0.094691,-0.038200,0.000818,0.021874,0.022685,-0.065212,0.034924,-0.260864
precipitacion_atlantico,-0.075014,0.017710,1.000000,0.013352,0.119168,0.019529,0.052842,0.036497,0.085676,0.116575,0.119205,0.230301,0.204079,0.060673,0.057876,0.053496,0.012250,0.022535,0.051031,0.137871,0.114213,-0.050822,-0.019426,0.163391,0.133536,-0.045853,0.117832,-0.168199,0.006932,-0.164992,0.070169,0.007875,-0.124734,-0.181007,0.068818,-0.023108,-0.049335,0.028422,0.089177,0.041105,-0.052018,0.086999,-0.092026,-0.067412,-0.184702,0.095447,-0.082236,0.115467,-0.007299
precipitacion_bogota,0.106463,0.045715,0.013352,1.000000,0.074195,0.104728,0.117971,0.109118,-0.002043,0.059264,0.215744,0.002575,0.242182,0.286513,0.285836,0.110124,0.033521,0.332314,0.235644,-0.014192,0.075289,0.072888,0.037687,0.434412,-0.056657,-0.061607,-0.010421,-0.036763,-0.116177,-0.068183,-0.121085,-0.107200,-0.085909,-0.045204,-0.034626,-0.205303,0.001194,-0.044992,-0.076165,0.032972,-0.155177,0.010893,-0.152004,-0.100928,-0.036242,-0.082595,-0.157813,0.252189,-0.042113
precipitacion_bolivar,-0.033369,0.247388,0.119168,0.074195,1.000000,0.063806,0.091918,0.142740,-0.004278,0.073121,0.167940,0.138250,0.235722,0.089413,0.090713,0.041052,-0.079292,0.048677,0.130465,0.120488,0.340527,0.145462,0.047074,0.252365,0.007794,0.158988,0.003807,-0.218628,-0.121354,-0.243621,-0.077080,-0.091759,-0.218361,-0.152417,-0.025532,-0.209518,-0.010243,-0.033582,-0.007320,0.043375,-0.110049,-0.008454,-0.201869,-0.137354,-0.191749,0.036608,-0.199072,0.191135,-0.166742
precipitacion_caqueta,-0.022714,0.209292,0.019529,0.104728,0.063806,1.000000,0.229139,0.085191,-0.036104,0.146477,0.223140,-0.008876,0.110767,0.417098,0.149357,-0.000406,0.299014,0.024021,0.092406,0.127199,0.066551,0.079250,0.118803,0.213683,-0.023762,0.012223,0.018240,-0.053670,-0.117982,-0.081146,-0.298821,-0.139115,-0.066724,-0.103962,-0.060351,-0.249599,0.032954,0.092904,-0.182588,-0.056841,0.004882,-0.195364,-0.056612,-0.046891,-0.062502,-0.034095,-0.112341,0.051083,-0.007909
precipitacion_casanare,-0.013511,0.208583,0.052842,0.117971,0.091918,0.229139,1.000000,0.018447,0.198906,0.141937,0.223296,0.023034,0.122743,0.234224,-

Se considera una Alta correlación entre variables a valores superiores a 0.5 o menores a -0.5, esto puede causar multicolinealidad y afectar la eficiencia de la predicción de los modelos, por lo tanto, se van a analizar las variables que tienen esta alta correlación para realizar una selección de variables inicial.


In [14]:
# Filter the correlation matrix based on the threshold (e.g., >= 0.5 or <= -0.5)
threshold = 0.5
filtered_corr_matrix = scaled_df_corr.where(
    (scaled_df_corr >= threshold) | (scaled_df_corr <= -threshold)
)

# Display the filtered correlation matrix without NaN values
print("Filtered Correlation Matrix:")
filtered_corr_matrix.dropna(how="all", axis=0).dropna(how="all", axis=1)
filtered_corr_matrix = filtered_corr_matrix.where((filtered_corr_matrix != 1.0))
filtered_corr_matrix = filtered_corr_matrix.dropna(how="all", axis=0).dropna(
    how="all", axis=1
)
# filtered_corr_matrix.replace(np.nan,0,inplace=True)
print(filtered_corr_matrix.shape)
filtered_corr_matrix.style.background_gradient(cmap="coolwarm", vmin=-1, vmax=1)

Filtered Correlation Matrix:
(21, 21)


,precipitacion_narino,precipitacion_quindio,precipitacion_departamentos_represa,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas
precipitacion_narino,nan,nan,0.501732,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
precipitacion_quindio,nan,nan,0.552464,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
precipitacion_departamentos_represa,0.501732,0.552464,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
temp_ATLANTICO,nan,nan,nan,nan,nan,0.771084,nan,nan,0.788871,nan,nan,0.652543,0.539691,nan,0.500997,nan,0.584163,0.583045,0.698579,nan,0.673706
temp_BOGOTA,nan,nan,nan,nan,nan,0.528524,0.517955,0.573468,0.515835,nan,0.565593,nan,0.509748,nan,0.581196,nan,0.585461,0.536363,nan,nan,0.744079
temp_BOLIVAR,nan,nan,nan,0.771084,0.528524,nan,nan,nan,0.788266,nan,nan,0.538978,nan,nan,nan,nan,0.615749,0.603872,0.833672,nan,0.750406
temp_CAQUETA,nan,nan,nan,nan,0.517955,nan,nan,0.538340,nan,0.512527,nan,nan,nan,0.528460,nan,0.611406,nan,nan,nan,nan,nan
temp_CASANARE,nan,nan,nan,nan,0.573468,nan,0.538340,nan,0.519304,0.533743,0.587534,nan,nan,0.508185,nan,nan,nan,nan,nan,0.554263,0.605086
temp_CESAR,nan,nan,nan,0.788871,0.515835,0.788266,nan,0.519304,nan,nan,nan,0.645398,0.546709,nan,0.581796,nan,0.600023,0.594558,0.701178,nan,0.713415
temp_GUAINIA,nan,nan,nan,nan,nan,nan,0.512527,0.533743,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Para filtrar las columnas, se hace lo siguiente:

**Criterio de experto**: Se deja siempre a la temperatrua y precipitación promedio que tiene represas

**Análisis de Correlación Parcial**: Para los otros casos, se deja a la variable que tiene una mayor correlación parcial contra Y


In [15]:
list_to_drop = [
    "precipitacion_narino",
    "precipitacion_quindio",
    "temp_ATLANTICO",
    "temp_BOGOTA",
    "temp_BOLIVAR",
    "temp_CASANARE",
    "temp_CESAR",
    "temp_GUAVIARE",
    "temp_LA GUAJIRA",
    "temp_MAGDALENA",
    "temp_NORTE DE SANTANDER",
    "temp_QUINDIO",
    "temp_RISARALDA",
    "temp_SUCRE",
    "temp_META",
    "temp_CASANARE",
    "temp_PUTUMAYO",
    "temp_GUAINIA",
]

In [16]:
print("scaled df sin filtrar columnas ", scaled_df.shape)
print(
    "scaled df después de filtrar columnas con alta correlación",
    scaled_df.drop(columns=list_to_drop).shape,
)
scaled_df_filtered = scaled_df.drop(columns=list_to_drop)
scaled_df_filter_corr = scaled_df_filtered.corr()

# Filter the correlation matrix based on the threshold (e.g., >= 0.5 or <= -0.5)
threshold = 0.5
filtered_corr_matrix = scaled_df_filter_corr.where(
    (scaled_df_corr >= threshold) | (scaled_df_corr <= -threshold)
)

# Display the filtered correlation matrix without NaN values
print("Filtered Correlation Matrix:")
filtered_corr_matrix.dropna(how="all", axis=0).dropna(how="all", axis=1)
filtered_corr_matrix = filtered_corr_matrix.where((filtered_corr_matrix != 1.0))
filtered_corr_matrix = filtered_corr_matrix.dropna(how="all", axis=0).dropna(
    how="all", axis=1
)

print(filtered_corr_matrix.shape)
filtered_corr_matrix.style.background_gradient(cmap="coolwarm", vmin=-1, vmax=1)

scaled df sin filtrar columnas  (644, 49)
scaled df después de filtrar columnas con alta correlación (644, 32)
Filtered Correlation Matrix:
(0, 0)


In [17]:
scaled_df_filtered.columns

Index(['precipitacion_amazonas', 'precipitacion_arauca',
       'precipitacion_atlantico', 'precipitacion_bogota',
       'precipitacion_bolivar', 'precipitacion_caqueta',
       'precipitacion_casanare', 'precipitacion_cesar', 'precipitacion_choco',
       'precipitacion_guainia', 'precipitacion_guaviare',
       'precipitacion_la guajira', 'precipitacion_magdalena',
       'precipitacion_meta', 'precipitacion_norte de santander',
       'precipitacion_putumayo', 'precipitacion_risaralda',
       'precipitacion_san andres providencia', 'precipitacion_sucre',
       'precipitacion_vaupes', 'precipitacion_vichada',
       'precipitacion_departamentos_represa', 'temp_AMAZONAS', 'temp_ARAUCA',
       'temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
       'temp_CAQUETA', 'temp_CHOCO', 'temp_NARINO', 'temp_VICHADA',
       'temp_avg_represas', 'brent_value', 'TRM'],
      dtype='object')

## Exportación de la data limpia sin lags


In [18]:
print(scaled_df_filtered.shape)
scaled_df_filtered.head()

(644, 32)


,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_CAQUETA,temp_CHOCO,temp_NARINO,temp_VICHADA,temp_avg_represas,brent_value,TRM
0,-0.509005,0.759339,0.139390,-0.557688,4.888002,-0.530775,-0.284457,-0.030483,0.599807,-0.518613,-0.661851,0.194869,1.081003,-0.330961,-0.425494,-0.641070,-0.197754,-0.290752,0.367185,1.680616,-0.382756,-0.103180,0.550670,3.036752,0.940739,0.503965,0.994447,1.267949,0.323702,-0.114337,-0.917312,-1.339552
1,-0.593583,1.622908,-0.161734,-0.569948,5.703726,-0.437031,-0.355927,0.273823,1.312841,-0.217009,0.073705,0.296772,0.108929,-0.280157,-0.533210,-0.651153,-0.023560,-0.206723,0.358932,0.541119,-0.356003,0.217245,0.750227,3.234346,0.864493,0.594068,0.773896,1.044002,1.159376,0.619803,-0.917312,-1.423606
2,-0.538867,1.559410,-0.248126,-0.567816,3.445073,-0.527751,-0.198452,-0.324530,0.818475,-0.402611,-0.450652,-0.226074,0.036084,-0.400241,-0.258923,-0.615863,-0.569591,0.248027,0.433208,1.436738,-0.420209,-0.115418,0.359982,3.357842,0.902616,0.670510,1.083718,1.926640,1.159376,0.950203,-0.917312,-1.423606
3,-0.580600,1.718155,-0.183251,-0.287443,3.802742,-0.300951,0.339387,-0.336497,0.724237,-0.472212,-0.159342,-0.231788,0.155604,0.027750,-0.288906,-0.504952,-0.455695,0.416086,0.016436,0.873297,0.532186,0.289020,-0.606760,3.370191,0.063912,0.470931,1.293766,1.612333,0.057805,0.723472,-0.761153,-1.423606
4,0.016831,2.981760,0.521690,-0.560886,0.362265,0.158697,-0.180160,0.179796,0.586998,-0.565013,0.741289,-0.272739,2.881082,2.346292,0.248564,1.188971,-0.325049,-0.266038,0.338299,2.731812,-0.281095,1.137304,-0.424941,3.036752,0.407018,-0.016688,-0.418129,0.717358,-0.701899,-0.197147,-0.614059,-1.411032


In [19]:
scaled_df_filtered["Date"] = df_merged["Date"]
scaled_df_filtered["energy_price"] = df_merged["energy_price"]
scaled_df_filtered.to_csv(
    "processed_tables/merged_without_lags_represas_criterio.csv", index=False
)